- This notebook finds which url for football players can be webscraped.

In [1]:
import requests
import re

from bs4 import BeautifulSoup as bs

import time
import random

import pandas as pd

import numpy as np

import signal

import os

import shutil

In [2]:
years = range(2018, 2024)

In [3]:
team_names=[
    '49ers',
    'Bears',
    'Bengals',
    'Broncos',
    'Browns',
    'Buccaneers',
    'Bills',
    'Cardinals',
    'Chargers',
    'Chiefs',
    'Colts',
    'Commanders',
    'Cowboys',
    'Dolphins',
    'Eagles',
    'Falcons',
    'Giants',
    'Jaguars',
    'Jets',
    'Lions',
    'Packers',
    'Panthers',
    'Patriots',
    'Raiders',
    'Rams',
    'Ravens',
    'Saints',
    'Seahawks',
    'Steelers',
    'Texans',
    'Titans',
    'Vikings'
]

In [4]:
def get_player_li (team: str, year: int):

    roster = pd.read_csv('NFL_Data_{}/{}/{}_roster'.format(year, team, team))

    roster = roster[:-1]

    #reset index
    roster.reset_index(drop=['index'], inplace=True)

    #get list of players
    players_li = list(roster['Player'].values)[:-1]
    
    return players_li

In [5]:
#loop through player_name list to get each player url
#url_number to differentiates similar players similar url names
#year to select which season

### this uses the first 2 characters of the first name
def player_url_func_1(player: str, url_number: int, year: int):
    
    '''
    player = str search for the player you're looking for
    url_number = int the number to distinguish between similar player names
    year = int which year you're searching in
    '''
    
    
    #split the names
    if player.isalpha() == False:
        
        #delete ' from names
        name_replace = player.replace("'", "")

        #leave out anything other than names
        split_name = name_replace.split()[:2]
            
            
    else:
        split_name = player.split()

    #players last name first character
    player_ln_1c = split_name[1][0]
    
    #players first two characters of first name
    player_fn_2c = split_name[0][:2]
        
    player_fn_2c.strip()

    player_fn_2c.replace('.','')
        
    #first four characters of last name
    if len(split_name[1]) < 4:
        player_ln_4c = split_name[1]+'x'
        
    else:
        player_ln_4c = split_name[1][:4]


    #the url
    #in order, the brackets to fill in
        #fist character from last name
        #first four characters from last name
        #first two characters of first name
        # number code for to differentiate similar url because it is based off names/initial
        #+1 when name doesn't match
        #year
    url = 'http://www.pro-football-reference.com/players/{}/{}{}{}/gamelog/{}/advanced/'

    #for the url code to differentiate names
    url_no = '{:02d}'.format(url_number)

    #url code
    url_requests = url.format(player_ln_1c, player_ln_4c, player_fn_2c, url_no, year)

    return url_requests

In [6]:
#loop through player_name list to get each player url
#url_number to differentiates similar players similar url names
#year to select which season

### this uses the first 3 characters of the first name
def player_url_func_2(player: str, url_number: int, year: int):
   
    '''
    Some url use a different number of characters from the player's name, to create up the url. 
    
    player = str search for the player you're looking for
    url_number = int the number to distinguish between similar player names
    year = int which year you're searching in
    '''
    
    #split the names
    if player.isalpha() == False:

        #delete ' from names
        name_replace = player.replace("'", "")

        #leave out anything other than names
        split_name = name_replace.split()[:2]

    else:
        split_name = player.split()

    #players last name first character
    player_ln_1c = split_name[1][0]

    #players first three characters of first name
    player_fn_3c = split_name[0][:3]


    #first four characters of last name
    if len(split_name[1]) < 4:
        player_ln_4c = split_name[1]+'xx'
    else:
        player_ln_4c = split_name[1][:5]


    #the url
    #in order, the brackets to fill in
        #fist character from last name
        #first four characters from last name
        #first two characters of first name
        # number code for to differentiate similar url because it is based off names/initial
            #+1 when name doesn't match
        #year
    url = 'http://www.pro-football-reference.com/players/{}/{}{}{}/gamelog/{}/advanced/'

    #for the url code to differentiate names
    url_no = '{:02d}'.format(url_number)

    #url code
    url_requests = url.format(player_ln_1c, player_ln_4c, player_fn_3c, url_no, year)

    return url_requests

In [7]:
# 5/27/2024 maybe causing issues with being abel to scrape tables
# becasue the name is wrong when being scrapped therefore returns an error
# and passes the webscrape function
def webscrape_player_name(url: str):
    '''
    Webscrape the for the player's name by generated url.
    
    url = str webscrapes url for player's name
    '''
    
    #request html
    html = requests.get(url).text

    time.sleep(random.randrange(5,7))

    #parse html
    soup = bs(html, 'html.parser')

    #find header for player name
    h1 = soup.find_all('h1')[0]


    try:
        #get player name
        span = h1.find_all('span')[0]

        name = span.contents[0]

    except:
        name = False


    #maybe replace the '.' in names would get more matches
    #will have to test later 5/2/2024

    return name

In [8]:
# goes with player_url_func_1
def get_player_url_1(search_player:str, year:int):
    
    '''
    Search for the correct player from the roster. 
    
    search_player = str Player you are searching for
    year = int The year of the roster youre using
    '''

    url_number_li = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 20, 21, 25, 44, 98, 99, 'n/a']
    
    
    for url_number in url_number_li:
        try:
            
            #url for initial search for player name on url
            search_player_url = player_url_func_1(player=search_player, url_number=url_number, year=year)
            
            player1 = webscrape_player_name(url=search_player_url)
            
            # matcing which
            if player1 != search_player:
                pass
            
            else:
                break
        
        except:
            return 'n/a ' + search_player
        
    return search_player_url

In [9]:
# goes with player_url_func_2
def get_player_url_2(search_player:str, year:int):
    url_number= 1
    
    try:
        #url for initial search for player name on url
        search_player_url = player_url_func_2(player=search_player, url_number=url_number, year=year)
            
        #player scraped from url    
        player1 = webscrape_player_name(url=search_player_url)
            
            # matcing which
        if player1 != search_player:
            return 'n/a ' + search_player
        
    except:
        return 'n/a ' + search_player
        

    return search_player_url

In [10]:
# get final and correct url
##### maybe###########
def final_url(player: str, year: int):
    '''
    Connecting all the functions. 
    '''
    try:
        get_player_url = get_player_url_1(search_player=player, year=year)
        
        time.sleep(random.randrange(5,7))

        if get_player_url == 'n/a '+player:
            get_player_url = get_player_url_2(search_player=player, year=year)

        
    except:
        return 'n/a ' + search_player

        
        
    return get_player_url

In [11]:
#uses determined url to webscrape player adv log table
# import as csv
def webscrape_player_adv_log_table(player_url: str, team: str, year: int):
    '''
    Retruns dataframe and player name
    '''
    try:
        #scrape table
        df_tables = pd.read_html(player_url)
        
        time.sleep(random.randrange(5,7))
        
        if len(df_tables) > 1:
            #numer to label different tables fromw same player
            no = 0
            for table in df_tables:
                    
                df = table.droplevel(level=0, axis=1)

                #webscrape name to compare to roster to see if correct player
                name = webscrape_player_name(player_url)

                df['Year'] = [year]*len(df)
                df['Team'] = [team]*len(df)

                
                #to csv
                #to local file
                df.to_csv('NFL_Data_{}/{}/Players Log/{}_{}'.format(year, team, name, no))
                    
                no+=1
                    
        else:
            df = df_tables[0].droplevel(level=0, axis=1)

            #get name
            name = webscrape_player_name(player_url)

            df['Year'] = [year]*len(df)
            df['Team'] = [team]*len(df)

            #to csv
            df.to_csv('NFL_Data_{}/{}/Players Log/{}'.format(year, team, name))

    except:
        pass

    try:
        return df
    except:
        return

In [12]:
stop

NameError: name 'stop' is not defined

# Get Player Adv Log URL

In [ ]:
#run one year at a time
# gets each player adv log url then webscrapes table

In [13]:
years = range(2018,2024)

In [14]:
# webscrape player logs
n_a_player_dict_2024 = {}

player_url_li = {'URL': [], 'Team': [], 'Year': []}
for year in years:
    for team in team_names:
        print(year, team)
        players_li = get_player_li(team=team, year=year)
        for player in players_li:    
            #get url
            adv_log_player_url = final_url(player=player, year=year)
    
            player_url_li.append(adv_log_player_url)
    
            #imports table to csv file
            webscrape_player_adv_log_table(player_url=adv_log_player_url, team=team, year=year)
    
            print(adv_log_player_url)

            player_url_li['URL'].append(adv_log_player_url)
            player_url_li['Team'].append(team)
            player_url_li['Year'].append(year)
            
            if adv_log_player_url[:3] == 'n/a':
                n_a_player_dict_2024.update({adv_log_player_url: ['', year, team]})

Bills
http://www.pro-football-reference.com/players/A/AlexLo99/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/A/AlleJo00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/A/AndeDe00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/B/BarkMa00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/B/BenjKe00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/B/BodiRu00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/B/BoetIk00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/B/BojoCo00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/B/BoldVi00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/B/BushRa00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/C/ClayCh00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/C/CrooJa00/gamelog/2018/advanced/
http://www.pro-football-reference.

C:\Users\Family\AppData\Local\Temp\ipykernel_16548\682142164.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_tables = pd.read_html(player_url)
C:\Users\Family\anaconda3\Lib\site-packages\pandas\io\html.py:666: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(udoc, features="html5lib", from_encoding=from_encoding)


n/a Zay Jones
http://www.pro-football-reference.com/players/K/KerlJe00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/L/LaceDe00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/L/LawsSh00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/L/LeexKh00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/L/LewiRy01/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/L/LotuSt00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/L/LoveMi00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/M/MarlDe00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/M/McClRa00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/M/McCoLe01/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/M/McDeCo00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/M/McKeIs00/gamelog/2018/advanced/
http://www.pro-football-re

C:\Users\Family\AppData\Local\Temp\ipykernel_16548\682142164.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_tables = pd.read_html(player_url)
C:\Users\Family\anaconda3\Lib\site-packages\pandas\io\html.py:666: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(udoc, features="html5lib", from_encoding=from_encoding)


n/a A.J. Epenesa
http://www.pro-football-reference.com/players/F/FeliJo00/gamelog/2020/advanced/
http://www.pro-football-reference.com/players/F/FergRe00/gamelog/2020/advanced/
http://www.pro-football-reference.com/players/F/FordCo00/gamelog/2020/advanced/
http://www.pro-football-reference.com/players/G/GillRe00/gamelog/2020/advanced/
http://www.pro-football-reference.com/players/H/HughJe99/gamelog/2020/advanced/
http://www.pro-football-reference.com/players/H/HydeMi00/gamelog/2020/advanced/
http://www.pro-football-reference.com/players/J/JackDa02/gamelog/2020/advanced/
http://www.pro-football-reference.com/players/J/JeffQu00/gamelog/2020/advanced/
http://www.pro-football-reference.com/players/J/JohnDa07/gamelog/2020/advanced/
http://www.pro-football-reference.com/players/J/JohnJa11/gamelog/2020/advanced/
http://www.pro-football-reference.com/players/J/JohnTa01/gamelog/2020/advanced/
http://www.pro-football-reference.com/players/J/JoneTa00/gamelog/2020/advanced/


...

n/a Deonte Harty
http://www.pro-football-reference.com/players/H/HydeMi00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/I/IngrJa00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/I/IsabAn00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/J/JackDa02/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/J/JohnTa01/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/J/JohnTy02/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/J/JonaKi00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/J/JoneDa04/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/J/JoseLi99/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/K/KincDa00/gamelog/2023/advanced/


C:\Users\Family\AppData\Local\Temp\ipykernel_16548\682142164.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_tables = pd.read_html(player_url)
C:\Users\Family\anaconda3\Lib\site-packages\pandas\io\html.py:666: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(udoc, features="html5lib", from_encoding=from_encoding)


n/a A.J. Klein
http://www.pro-football-reference.com/players/K/KnoxDa00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/L/LawsSh00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/L/LewiCa00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/M/MartSa01/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/M/MataTy00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/M/McGoCo00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/M/MilaMa00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/M/MillVo00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/M/MorrQu00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/M/MorsMi00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/M/MurrLa00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/N/NealSi00/gamelog/2023/advanced/
http://www.pro-football-r

C:\Users\Family\AppData\Local\Temp\ipykernel_16548\682142164.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_tables = pd.read_html(player_url)
C:\Users\Family\anaconda3\Lib\site-packages\pandas\io\html.py:666: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(udoc, features="html5lib", from_encoding=from_encoding)


n/a Ryan Van Demark
http://www.pro-football-reference.com/players/V/VickKe00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/W/WhitTr01/gamelog/2023/advanced/


###### player_url_li

In [ ]:
df_url = pd.DataFrame(data=player_url_li, columns=[f'URL_{year}'])
df_url

In [ ]:
df_url.to_csv(f'URL_Players')

In [25]:
n_a_player_dict_2024

In [ ]:
stop

# Players Not Webscraped

- This section retries webscraping some players that the previous funcitons couldn't capture they player url, so I found each url so they can be directly put in the url so it can webscrape the table. Some players do not have a table.

In [17]:
team_names_dict={
    'SFO': '49ers', 
    'ARI': 'Cardinals',  
    'ATL': 'Falcons', 
    'BAL': 'Ravens', 
    'BUF': 'Bills',  
    'CAR': 'Panthers',  
    'CHI': 'Bears', 
    'CIN': 'Bengals', 
    'CLE': 'Browns',  
    'DAL': 'Cowboys', 
    'DEN': 'Broncos', 
    'DET': 'Lions',  
    'GNB': 'Packers',  
    'HOU': 'Texans', 
    'IND': 'Colts',  
    'JAX': 'Jaguars',  
    'KAN': 'Chiefs',  
    'LVR': 'Raiders',  
    'LAC': 'Chargers',  
    'LAR': 'Rams', 
    'MIA': 'Dolphins',  
    'MIN': 'Vikings',  
    'NWE': 'Patriots', 
    'NOR': 'Saints', 
    'NYG': 'Giants', 
    'NYJ': 'Jets',  
    'PHI': 'Eagles', 
    'PIT': 'Steelers',  
    'SEA': 'Seahawks', 
    'TAM': 'Buccaneers', 
    'TEN': 'Titans', 
    'WAS': 'Commanders' 
}

In [18]:
# 5/27/2024 completed webscraping player tables
n_a_player_dict_2018 = {
    'n/a Adam Jones': 'J/JonePa20',
    'n/a T.J. Carrie': 'C/CarrTr00',
    'n/a J.C. Tretter': 'T/TretJC00', # no table
    'n/a Adarius Taylor': 'G/GlanAd00', 
    'n/a Bobo Wilson': 'W/WilsJe00', 
    'n/a Alex Ellis': 'E/ElliAl01', # no tables
    'n/a Shaquille Leonard': 'L/LeonDa00',
    'n/a Ha Ha Clinton-Dix': 'C/ClinHa00',
    'n/a D.J. Swearinger': 'S/SweaDJ00',
    'n/a Walt Aikens': 'A/AikeWa00',
    'n/a Chase Allen': 'A/AlleCh02',
    'n/a T.J. McDonald': 'M/McDoTJ00',
    'n/a B.W. Webb': 'W/WebbBW00',
    'n/a D.J. Hayden': 'H/HaydDJ00',
    "n/a James O'Shaughnessy": "O/O'ShJa00",
    'n/a Robbie Chosen': 'A/AndeRo04',
    'n/a Equanimeous St. Brown': 'S/ST.BEq00',
    'n/a Greg Van Roten': 'V/VanRGr00', # no tables
    'n/a Kyle Van Noy': 'V/VanNKy00', 
    'n/a Tank Carradine': 'C/CarrCo00', # no table
    'n/a Gerald Everett': 'E/EverGe00',
    'n/a Todd Gurley': 'G/GurlTo01',
    'n/a A.J. Klein': 'K/KleiAJ00', 
    'n/a D.J. Fluker': 'F/FlukDJ00', # no tables
    'n/a Lano Hill': 'H/HillDe02', 
    'n/a Bud Dupree': 'D/DuprAl00',
    'n/a Buddy Howell': 'H/HoweGr00',
    'n/a Johnathan Joseph': 'J/JoseJo20',
    'n/a Peter Kalambayi': 'K/KalaPe00',
    'n/a Delanie Walker': 'W/WalkHu00',
}

In [19]:
n_a_player_dict_2019 = {
    'n/a Azeez Al-Shaair': 'A/AlShAz00',
    'n/a Ha Ha Clinton-Dix': 'C/ClinHa00',
    'n/a Dre Kirkpatrick': 'K/KirkDr00',
    'n/a Stanley Morgan Jr.': 'M/MorgSt02',
    'n/a B.W. Webb': 'W/WebbBW00',
    'n/a T.J. Carrie': 'C/CarrTr00',
    'n/a J.T. Hassell': 'H/HassJT00',
    'n/a Adarius Taylor': 'G/GlanAd00',
    'n/a J.C. Tretter': 'T/TretJC00', # no table
    'n/a Sean Murphy-Bunting': 'B/BuntSe00' ,
    'n/a Bobo Wilson': 'W/WilsJe00',
    'n/a D.J. Swearinger': 'S/SweaDJ00',
    'n/a Shaquille Leonard': 'L/LeonDa00',
    'n/a E.J. Speed': 'S/SpeeEJ00',
    'n/a Tank Carradine': 'C/CarrCo00',
    'n/a Jamal Perry': 'W/WiltJo01',
    'n/a Andrew Van Ginkel': 'V/VanGAn00',
    'n/a T.J. Edwards': 'E/EdwaTJ02',
    'n/a Brandon Graham': 'G/GrahBr99',
    'n/a Kamu Grugier-Hill': 'G/GrugKa00',
    'n/a Daeshon Hall': 'H/HallDa01',
    'n/a D.J. Hayden': 'H/HaydDJ00',
    "n/a James O'Shaughnessy": "O/O'ShJa00",
    'n/a Brandon Rusnak': 'W/WatsBr01',
    'n/a Robbie Chosen': 'A/AndeRo04',
    'n/a T.J. Hockenson': 'H/HockTJ00',
    'n/a C.J. Moore': 'M/MoorCJ01',
    'n/a Greg Van Roten': 'V/VanRGr00', # no table
    'n/a Kyle Van Noy': 'V/VanNKy00',
    'n/a C.J. Gardner-Johnson': 'G/GardCh00',
    'n/a Deonte Harty': 'H/HarrDe07',
    'n/a A.J. Klein': 'K/KleiAJ00',
    'n/a L.J. Collier': 'C/CollLJ00',
    'n/a D.J. Fluker': 'F/FlukDJ00', # no table
    'n/a Lano Hill': 'H/HillDe02',
    'n/a D.K. Metcalf': 'M/MetcDK00',
    'n/a Bud Dupree': 'D/DuprAl00',
    'n/a Buddy Howell': 'H/HoweGr00',
    'n/a A.J. Brown': 'B/BrowAJ00',
    'n/a Delanie Walker': 'W/WalkHu00',
    'n/a Marcus Sherels': 'S/SherMa20',
    'n/a Cameron Smith': 'S/SmitCa01',
    'n/a Harrison Smith': 'S/SmitHa00'
}

In [20]:
n_a_player_dict_2020 = {
    'n/a Azeez Al-Shaair': 'A/AlShAz00',
    'n/a Kwon Alexander': 'A/AlexKw00',
    'n/a P.J. Locke': 'L/LockPJ00',
    'n/a J.C. Tretter': 'T/TretJC00', # no table
    'n/a Sean Murphy-Bunting': 'B/BuntSe00' ,
    'n/a A.J. Epenesa': 'E/EpenAJ00',
    'n/a A.J. Klein': 'K/KleiAJ00',
    'n/a K.J. Hill': 'H/HillKJ00',
    'n/a Tyree St. Louis': 'S/StLoTy01', # no table
    'n/a Willie Gay Jr.': 'G/GayJWi00',
    'n/a T.J. Carrie': 'C/CarrTr00',
    'n/a Shaquille Leonard': 'L/LeonDa00',
    'n/a Al-Quadin Muhammad': 'M/MuhaAl00',
    'n/a E.J. Speed': 'S/SpeeEJ00',
    'n/a Hunter Niswander': 'N/NiswJa00', # no table
    'n/a Jamal Perry': 'W/WiltJo01',
    'n/a Andrew Van Ginkel': 'V/VanGAn00',
    'n/a Kyle Van Noy': 'V/VanNKy00',
    'n/a T.J. Edwards': 'E/EdwaTJ02',
    'n/a Sua Opeta': 'O/Opetlo00',
    'n/a Prince Tega Wanogho': 'W/WanoTe00', # no table
    'n/a T.J. Brunson': 'B/BrunTJ00',
    'n/a Levine Toilolo': 'T/ToilLe00',
    'n/a Dalvin Tomlinson': 'T/TomlDa00',
    'n/a D.J. Hayden': 'H/HaydDJ00',
    'n/a C.J. Henderson': 'H/HendCJ00',
    "n/a James O'Shaughnessy": "O/O'ShJa00",
    'n/a Brandon Rusnak': 'W/WatsBr01',
    'n/a J.T. Hassell': 'H/HassJT00',
    'n/a Greg Van Roten': 'V/VanRGr00',
    'n/a T.J. Hockenson': 'H/HockTJ00',
    'n/a C.J. Moore': 'M/MoorCJ01',
    'n/a Adrian Amos': 'A/AmosAd00',
    'n/a Equanimeous St. Brown': 'S/ST.BEq00',
    'n/a Robbie Chosen': 'A/AndeRo04',
    'n/a Adarius Taylor': 'G/GlanAd00',
    'n/a P.J. Walker': 'W/WalkPh00',
    'n/a J.J. Taylor': 'T/TaylJJ01',
    'n/a J.K. Dobbins': 'D/DobbJK00',
    'n/a D.J. Fluker': 'F/FlukDJ00', # no table
    'n/a C.J. Gardner-Johnson': 'G/GardCh00',
    'n/a Deonte Harty': 'H/HarrDe07',
    'n/a D.J. Swearinger': 'S/SweaDJ00',
    'n/a L.J. Collier': 'C/CollLJ00',
    'n/a Lano Hill': 'H/HillDe02',
    'n/a D.K. Metcalf': 'M/MetcDK00',
    'n/a Bud Dupree': 'D/DuprAl00',
    'n/a Buddy Howell': 'H/HoweGr00',
    'n/a A.J. Brown': 'B/BrowAJ00',
    'n/a K.J. Osborn': 'O/OsboKJ00',
    'n/a D.J. Wonnum': 'W/WonnDJ00'
}


In [21]:
n_a_player_dict_2021 = {
    'n/a Azeez Al-Shaair': 'A/AlShAz00',
    'n/a Jamar Johnson': 'J/JohnJa15', # no table
    'n/a P.J. Locke': 'L/LockPJ00',
    'n/a J.C. Tretter': 'T/TretJC00', # no table
    'n/a K.J. Britt': 'B/BritKJ00',
    'n/a Sean Murphy-Bunting': 'B/BuntSe00' ,
    'n/a Boogie Basham': 'B/BashCa00',
    'n/a A.J. Epenesa': 'E/EpenAJ00',
    'n/a A.J. Klein': 'K/KleiAJ00',
    'n/a K.J. Hill': 'H/HillKJ00',
    'n/a Willie Gay Jr.': 'G/GayJWi00',
    'n/a Prince Tega Wanogho': 'W/WanoTe00', # no table
    'n/a T.J. Carrie': 'C/CarrTr00',
    'n/a Shaquille Leonard': 'L/LeonDa00',
    'n/a E.J. Speed': 'S/SpeeEJ00',
    'n/a D.J. Hayden': 'H/HaydDJ00', # no table
    'n/a Benjamin St-Juste': 'S/StJuBe00',
    'n/a Jamal Perry': 'W/WiltJo01',
    'n/a Andrew Van Ginkel': 'V/VanGAn00',
    'n/a T.J. Edwards': 'E/EdwaTJ02',
    'n/a Sua Opeta': 'O/Opetlo00', # no table
    'n/a Deion Jones': 'J/JoneDe01',
    'n/a Jarren Williams': 'W/WillJa09',
    'n/a Leonard Williams': 'W/WillLe02',
    'n/a C.J. Henderson': 'H/HendCJ00',
    "n/a James O'Shaughnessy": "O/O'ShJa00",
    'n/a Brandon Rusnak': 'W/WatsBr01',
    'n/a D.J. Montgomery': 'M/MontDJ01',
    'n/a Greg Van Roten': 'V/VanRGr00', # no table
    'n/a T.J. Hockenson': 'H/HockTJ00',
    'n/a C.J. Moore': 'M/MoorCJ01',
    'n/a Amon-Ra St. Brown': 'S/STxxAm00',
    'n/a Equanimeous St. Brown': 'S/ST.BEq00',
    'n/a Robbie Chosen': 'A/AndeRo04',
    'n/a C.J. Saunders': 'S/SaunCJ00',
    'n/a P.J. Walker': 'W/WalkPh00',
    'n/a J.J. Taylor': 'T/TaylJJ01',
    'n/a Kyle Van Noy': 'V/VanNKy00',
    'n/a Ha Ha Clinton-Dix': 'C/ClinHa00', # no table
    'n/a Blake Countess': 'C/CounBl00', # no table
    'n/a David Edwards': 'E/EdwaDa01',
    'n/a Buddy Howell': 'H/HoweGr00',
    'n/a J.J. Koski': 'K/KoskJJ00', # no table
    'n/a Odafe Oweh': 'O/OwehJa00',
    'n/a C.J. Gardner-Johnson': 'G/GardCh00',
    'n/a Deonte Harty': 'H/HarrDe07',
    'n/a L.J. Collier': 'C/CollLJ00',
    'n/a D.K. Metcalf': 'M/MetcDK00',
    'n/a A.J. Brown': 'B/BrowAJ00',
    'n/a Bud Dupree': 'D/DuprAl00',
    'n/a K.J. Osborn': 'O/OsboKJ00',
    'n/a T.J. Smith': 'S/SmitTJ00',
    'n/a D.J. Wonnum': 'W/WonnDJ00'
}

In [22]:
n_a_player_dict_2022 = {
    'n/a Azeez Al-Shaair': 'A/AlShAz00',
    'n/a A.J. Klein': 'K/KleiAJ00',
    'n/a Equanimeous St. Brown': 'S/ST.BEq00',
    'n/a A.J. Thomas': 'T/ThomAJ00',
    'n/a Kindle Vildor': 'V/VildKi00',
    'n/a P.J. Locke': 'L/LockPJ00',
    'n/a K.J. Britt': 'B/BritKJ00',
    'n/a Sean Murphy-Bunting': 'B/BuntSe00',
    'n/a J.J. Russell': 'R/RussJJ00',
    'n/a Boogie Basham': 'B/BashCa00',
    'n/a C.J. Brewer': 'B/BrewCJ00',
    'n/a A.J. Epenesa': 'E/EpenAJ00',
    'n/a Jaquan Johnson': 'J/JohnJa11',
    'n/a Taron Johnson': 'J/JohnTa01',
    'n/a Greg Van Roten': 'V/VanRGr00', # no table
    'n/a Robbie Chosen': 'A/AndeRo04',
    'n/a Wyatt Davis': 'D/DaviWy00', # no table
    'n/a Kyle Van Noy': 'V/VanNKy00',
    'n/a Willie Gay Jr.': 'G/GayJWi00',
    'n/a Prince Tega Wanogho': 'W/WanoTe00', # no table
    'n/a Shaquille Leonard': 'L/LeonDa00',
    'n/a E.J. Speed': 'S/SpeeEJ00',
    'n/a Benjamin St-Juste': 'S/StJuBe00',
    'n/a Andrew Van Ginkel': 'V/VanGAn00',
    'n/a A.J. Brown': 'B/BrowAJ00',
    'n/a T.J. Edwards': 'E/EdwaTJ02',
    'n/a C.J. Gardner-Johnson': 'G/GardCh00',
    'n/a Sua Opeta': 'O/Opetlo00', # no table
    'n/a D.J. Davidson': 'D/DaviDJ00', 
    'n/a T.J. Hockenson': 'H/HockTJ00',
    'n/a C.J. Moore': 'M/MoorCJ01',
    'n/a Amon-Ra St. Brown': 'S/STxxAm00',
    'n/a T.J. Carrie': 'C/CarrTr00',
    'n/a C.J. Henderson': 'H/HendCJ00',
    'n/a P.J. Walker': 'W/WalkPh00',
    'n/a J.J. Taylor': 'T/TaylJJ01',
    'n/a Jordan Meredith': 'M/MereJo00',  # no table
    'n/a T.J. Carter': 'C/CartTJ00',
    'n/a Cobie Durant': 'D/DuraDe01',
    'n/a J.K. Dobbins': 'D/DobbJK00',
    'n/a Odafe Oweh': 'O/OwehJa00',
    'n/a Damarion Williams': 'W/WillDa15',
    'n/a Deonte Harty': 'H/HarrDe07',
    'n/a L.J. Collier': 'C/CollLJ00',
    'n/a D.K. Metcalf': 'M/MetcDK00',
    'n/a Riq Woolen': 'W/WoolTa00',
    'n/a Bud Dupree': 'D/DuprAl00',
    'n/a K.J. Osborn': 'O/OsboKJ00',
    'n/a T.J. Smith': 'S/SmitTJ00',
    'n/a D.J. Wonnum': 'W/WonnDJ00'
}

In [23]:
n_a_player_dict_2023 = {
    'n/a T.J. Edwards': 'E/EdwaTJ02',
    'n/a Equanimeous St. Brown': 'S/ST.BEq00',
    'n/a A.J. Thomas': 'T/ThomAJ00', # no table
    'n/a P.J. Locke': 'L/LockPJ00',
    'n/a Javonte Williams': 'W/Willja10',
    'n/a P.J. Walker': 'W/WalkPh00',
    'n/a K.J. Britt': 'B/BritKJ00',
    'n/a J.J. Russell': 'R/RussJJ00',
    'n/a A.J. Epenesa': 'E/EpenAJ00',
    'n/a Deonte Harty': 'H/HarrDe07',
    'n/a A.J. Klein': 'K/KleiAJ00',
    'n/a Ryan Van Demark': 'V/VanDRy00', # no table
    'n/a L.J. Collier': 'C/CollLJ00',
    'n/a Willie Gay Jr.': 'G/GayJWi00',
    'n/a Prince Tega Wanogho': 'W/WanoTe00', # no table
    'n/a Sam Ehlinger': 'E/EhliSa00', # no table
    'n/a Shaquille Leonard': 'L/LeonDa00',
    'n/a D.J. Montgomery': 'M/MontDJ01',
    'n/a E.J. Speed': 'S/SpeeEJ00',
    'n/a K.J. Henry': 'H/HenrKJ00',
    'n/a Quan Martin': 'M/MartJa04',
    'n/a Benjamin St-Juste': 'S/StJuBe00',
    'n/a T.J. Bass': 'B/BassTJ00',  # no table
    'n/a Robbie Chosen': 'A/AndeRo04',
    'n/a Andrew Van Ginkel': 'V/VanGAn00',
    'n/a A.J. Brown': 'B/BrowAJ00',
    'n/a Sua Opeta': 'O/Opetlo00', # no table
    'n/a Bud Dupree': 'D/DuprAl00',
    'n/a Boogie Basham': 'B/BashCa00',
    'n/a D.J. Davidson': 'D/DaviDJ00',
    'n/a C.J. Gardner-Johnson': 'G/GardCh00',
    'n/a Amon-Ra St. Brown': 'S/STxxAm00',
    'n/a Romeo Doubs': 'D/DoubRo00',
    'n/a Lukas Van Ness': 'V/VanNlu00',
    'n/a J.D. DiRenzo': 'D/DiReJD00', # no table
    'n/a C.J. Henderson': 'H/HendCJ00',
    'n/a Justin Houston': 'H/HousJu00',
    'n/a Chuba Hubbard': 'H/HubbCh01',
    'n/a Tyquan Thornton': 'T/ThorTy00',
    'n/a Greg Van Roten': 'V/VanRGr00', # no table
    'n/a Cobie Durant': 'D/DuraDe01',
    "n/a Tre'Vius Tomlinson": 'H/HodgTr00',
    'n/a J.K. Dobbins': 'D/DobbJK00',
    'n/a Odafe Oweh': 'O/OwehJa00',
    'n/a Kyle Van Noy': 'V/VanNKy00',
    'n/a Damarion Williams': 'W/WillDa15', # no table
    'n/a A.T. Perry': 'P/PerrAT00',
    'n/a D.K. Metcalf': 'M/MetcDK00',
    'n/a Riq Woolen': 'W/WoolTa00',
    'n/a Tank Dell': 'd/DellNa00',
    'n/a C.J. Stroud': 'S/StroCJ00',
    'n/a Azeez Al-Shaair': 'A/AlShAz00',
    'n/a Sean Murphy-Bunting': 'B/BuntSe00',
    'n/a T.J. Hockenson': 'H/HockTJ00',
    'n/a K.J. Osborn': 'O/OsboKJ00',
    'n/a T.J. Smith': 'S/SmitTJ00',
    'n/a Jaylin Williams': 'W/WillJa14',
    'n/a D.J. Wonnum': 'W/WonnDJ00'
}

In [24]:
n_a_player_dict_2024 = {
    
}

In [ ]:
n_a_player_dict_2024

In [ ]:
#fill new roster's na url with previous years so I have to search for less 
for dict in [n_a_player_dict_2023, n_a_player_dict_2022, n_a_player_dict_2021, n_a_player_dict_2020, n_a_player_dict_2019, n_a_player_dict_2018]:
    for k in dict.keys():
        if k in n_a_player_dict_2024.keys():
            print(k)
            n_a_player_dict_2024[k] = dict[k]

In [26]:
na_dicts = {
    2018:n_a_player_dict_2018,
    2019:n_a_player_dict_2019,
    2020:n_a_player_dict_2020,
    2021:n_a_player_dict_2021,
    2022:n_a_player_dict_2022,
    2023:n_a_player_dict_2023,
    2024:n_a_player_dict_2024
}

In [30]:
# re run na players

final_try_webscrape_table = {}

for year in na_dicts.keys():
    url_na_player_df = pd.read_csv(f'NFL_Data_{year}/URL_Players', index_col=0)
    
    for player_url in na_dicts[year].keys():
        print(player_url)
        
        time.sleep(random.randrange(7,12))
    
        #get name
        name = player_url[4:]
        
        try:
            
            url = "http://www.pro-football-reference.com/players/{}/gamelog/{}/advanced/"

            p_url = na_dicts[year][player_url]
            
            #url code
            url_requests = url.format(p_url, year)
            
            
            #scrape table
            df_tables = pd.read_html(url_requests)
                    
            if len(df_tables) > 1:
                #numer to label different tables fromw same player
                no = 0
                for table in df_tables:
                        
                    df = table.droplevel(level=0, axis=1)
            
                    df['Year'] = [year]*len(df)
            
                    team_abbrev = df['Tm'][0]
                
                    team = team_names_dict[team_abbrev]
                    
                    #to csv
                    #to local file
                    df.to_csv('NFL_Data_{}/{}/Players Log/{}_{}'.format(year, team, name, no))
                        
                    no+=1
                        
            else:
                df = df_tables[0].droplevel(level=0, axis=1)
                                
                df['Year'] = [year]*len(df)
                
                team_abbrev = df['Tm'][0]
                
                team = team_names_dict[team_abbrev]
                
                #to csv
                df.to_csv('NFL_Data_{}/{}/Players Log/{}'.format(year, team, name))
    
    
        except:
            print('***************No Table**************************', player_url)
            
            url = "http://www.pro-football-reference.com/players/{}/gamelog/{}/advanced/"
                       
            #url code
            url_requests = url.format(p_url, year)
    
            final_try_webscrape_table[player_url] = [p_url, url_requests, year]
    
            print(url_requests)

        url_df_convert = pd.DataFrame(data=[[url]], columns=[f'URL_{year}'])
        url_na_player_df = pd.concat([url_na_player_df, url_df_convert], axis=0)

    url_na_player_df = url_na_player_df.reset_index()
    url_na_player_df.to_csv(f'NFL_Data_{year}/URL_Players')

FileNotFoundError: [Errno 2] No such file or directory: 'NFL_Data_2024/URL_Players'

In [ ]:
#check these one by one
final_try_webscrape_table

In [ ]:
pd.read_csv('NFL_Data_2018/URL_Players', index_col=0)